In [1]:
import os
import subprocess
import tempfile
from pathlib import Path

In [2]:
### SET CWD TO REPO ROOT

os.chdir(Path.cwd().parent)
print("CWD now:", Path.cwd())

CWD now: /workspaces/LMUL-Hardware-Acceleration


In [3]:
def test_verilog_function(x, y, verilog_file='rtl/simple_function.v'):
    """
    Test Verilog function using iverilog (simpler, no cocotb needed)
    This is the EASIEST approach for Jupyter!
    """
    # Create a testbench
    testbench = f'''
`timescale 1ns/1ps

module tb;
    reg [7:0] x, y;
    wire [7:0] result;
    
    simple_function dut (
        .x(x),
        .y(y),
        .result(result)
    );
    
    initial begin
        x = {x};
        y = {y};
        #1;
        $display("%d", result);
        $finish;
    end
endmodule
'''
    
    # Write testbench
    with tempfile.NamedTemporaryFile(mode='w', suffix='.v', delete=False) as f:
        tb_file = f.name
        f.write(testbench)
    
    try:
        # Compile with iverilog
        out_file = '/tmp/sim.out'
        compile_result = subprocess.run(
            ['iverilog', '-o', out_file, verilog_file, tb_file],
            capture_output=True,
            text=True
        )
        
        if compile_result.returncode != 0:
            raise RuntimeError(f"Compilation failed:\n{compile_result.stderr}")
        
        # Run simulation
        sim_result = subprocess.run(
            ['vvp', out_file],
            capture_output=True,
            text=True
        )
        
        # Extract result
        result = int(sim_result.stdout.strip())
        return result
        
    finally:
        # Cleanup
        if os.path.exists(tb_file):
            os.remove(tb_file)
        if os.path.exists(out_file):
            os.remove(out_file)

In [4]:
def f(x, y):
    """Call Verilog function f(x,y) = x + y + 1"""
    return test_verilog_function(x, y)

In [5]:
print("Running tests...")
print(f"f(5, 10) = {f(5, 10)} (expected: 16)")
print(f"f(100, 50) = {f(100, 50)} (expected: 151)")
print(f"f(0, 0) = {f(0, 0)} (expected: 1)")
print(f"f(255, 0) = {f(255, 0)} (expected: 0)")

Running tests...
f(5, 10) = 16 (expected: 16)
f(100, 50) = 151 (expected: 151)
f(0, 0) = 1 (expected: 1)
f(255, 0) = 0 (expected: 0)
